# tss detection

In [ ]:
# #!/bin/bash
# #Input flags for batch files, do not run on notebook
# while getopts 'i:' flag; do
#   case "${flag}" in
#       i) BAMFiles="${OPTARG}";;
#       ?) echo "I don't know what flag is this" ;;
#            esac
#        done

In [1]:
#temporary assignment of input variables for notebook, remove or comment out in bash file
BAMFiles=input_bam_files/BAM_filesLUZ100

# function to determine peaks using termseq_peaks

In [2]:
# function to determine peaks using termseq_peaks
determinePeaks(){
    directoryBAMFiles=$1
    outputDirectory=$2
    ends=$3
    significanceLevel=$(4:-0.001)
    
    for file in $directoryBAMFiles/*.bam
    do
        # Extract file name of BAM file to use for other files
        filename_extended=${file##*/}
        searchstring=".sorted.bam"
        filename=${filename_extended%$searchstring*}

        # Create directory based on file name
        mkdir ${outputDirectory}
        mkdir ${outputDirectory}/${filename}

        if [[ ends=="5" ]]
        then
            # Get genome coverage of bam files for both + and - strands. Used for termseq_peaks
            bedtools genomecov \
            -ibam ${file} \
            -bga \
            -5 \
            -strand + > ${outputDirectory}/${filename}/${filename}.plus.bedgraph

            bedtools genomecov \
            -ibam ${file} \
            -bga \
            -5 \
            -strand - > ${outputDirectory}/${filename}/${filename}.minus.bedgraph
            elif [[ ends=="3" ]]
        then
            # Get genome coverage of bam files for both + and - strands. Used for termseq_peaks
            bedtools genomecov \
            -ibam ${file} \
            -bga \
            -3 \
            -strand + > ${outputDirectory}/${filename}/${filename}.plus.bedgraph

            bedtools genomecov \
            -ibam ${file} \
            -bga \
            -3 \
            -strand - > ${outputDirectory}/${filename}/${filename}.minus.bedgraph
        else
            echo "error: wrong ends flag"
        fi
        

        # Use termseq_peaks to get peaks  
        termseq_peaks ${outputDirectory}/${filename}/${filename}.plus.bedgraph \
        ${outputDirectory}/${filename}/${filename}.plus.bedgraph --peaks ${outputDirectory}/${filename}/${filename}.plus.peaks \
        --strand + -t $significanceLevel

        termseq_peaks ${outputDirectory}/${filename}/${filename}.minus.bedgraph \
        ${outputDirectory}/${filename}/${filename}.minus.bedgraph --peaks ${outputDirectory}/${filename}/${filename}.minus.peaks \
        --strand - -t $significanceLevel

        # Add coverage information
        bedtools intersect \
          -wao \
          -a ${outputDirectory}/${filename}/${filename}.plus.peaks.oracle.narrowPeak \
          -b ${outputDirectory}/${filename}/${filename}.plus.bedgraph \
          > ${outputDirectory}/${filename}/${filename}.plus.peaks.oracle.narrowPeak.counts

        bedtools intersect \
          -wao \
          -a ${outputDirectory}/${filename}/${filename}.minus.peaks.oracle.narrowPeak \
          -b ${outputDirectory}/${filename}/${filename}.minus.bedgraph \
          > ${outputDirectory}/${filename}/${filename}.minus.peaks.oracle.narrowPeak.counts

          # Add total mapped reads
          total_mapped=$(samtools view -c -F4 $directoryBAMFiles/$filename_extended)

        awk '{print $1, $2, $3, $4, $5, $6, $7, $8 ,$9, $10, $11, $12, $13, $14, $15, '$total_mapped'}' \
        ${outputDirectory}/${filename}/${filename}.plus.peaks.oracle.narrowPeak.counts \
        > ${outputDirectory}/${filename}/${filename}.plus.peaks.oracle.narrowPeak.counts.normalized


        awk '{print $1, $2, $3, $4, $5, $6, $7, $8 ,$9, $10, $11, $12, $13, $14, $15, '$total_mapped'}' \
        ${outputDirectory}/${filename}/${filename}.minus.peaks.oracle.narrowPeak.counts \
        > ${outputDirectory}/${filename}/${filename}.minus.peaks.oracle.narrowPeak.counts.normalized
done
}

# TSS detection and clustering

In [7]:
# peak calling
determinePeaks $BAMFiles tssLUZ100 "5" 0.05

mkdir: cannot create directory ‘tssLUZ100’: File exists
mkdir: cannot create directory ‘tssLUZ100/infectedControl-LUZ100_full-genome’: File exists
INFO:root:Unioning bedgraphs...
INFO:root:Averaging bedgraphs...
INFO:root:/home/yostrich/anaconda3/lib/python3.9/site-packages/peaklib/pybedtools.dwnst74i.tmp
INFO:root:Phage_LUZ100_full_genome
INFO:root:tssLUZ100/infectedControl-LUZ100_full-genome/infectedControl-LUZ100_full-genome.plus.bedgraph
INFO:root:Phage_LUZ100_full_genome
INFO:root:tssLUZ100/infectedControl-LUZ100_full-genome/infectedControl-LUZ100_full-genome.plus.bedgraph
INFO:root:Phage_LUZ100_full_genome
INFO:root:Merging peaks for /home/yostrich/anaconda3/lib/python3.9/site-packages/peaklib/pybedtools.i2pwk6d7.tmp and /home/yostrich/anaconda3/lib/python3.9/site-packages/peaklib/pybedtools.mss98ect.tmp
INFO:root:Calcluating IDR for /home/yostrich/anaconda3/lib/python3.9/site-packages/peaklib/pybedtools.i2pwk6d7.tmp and /home/yostrich/anaconda3/lib/python3.9/site-packages/peakli

INFO:root:Averaging bedgraphs...
INFO:root:/home/yostrich/anaconda3/lib/python3.9/site-packages/peaklib/pybedtools.i881maau.tmp
INFO:root:Phage_LUZ100_full_genome
INFO:root:tssLUZ100/infectedEnriched-LUZ100_full-genome/infectedEnriched-LUZ100_full-genome.minus.bedgraph
INFO:root:Phage_LUZ100_full_genome
INFO:root:tssLUZ100/infectedEnriched-LUZ100_full-genome/infectedEnriched-LUZ100_full-genome.minus.bedgraph
INFO:root:Phage_LUZ100_full_genome
INFO:root:Merging peaks for /home/yostrich/anaconda3/lib/python3.9/site-packages/peaklib/pybedtools.k_rv7lvx.tmp and /home/yostrich/anaconda3/lib/python3.9/site-packages/peaklib/pybedtools.vw5xfqog.tmp
INFO:root:Calcluating IDR for /home/yostrich/anaconda3/lib/python3.9/site-packages/peaklib/pybedtools.k_rv7lvx.tmp and /home/yostrich/anaconda3/lib/python3.9/site-packages/peaklib/pybedtools.vw5xfqog.tmp
Initial parameter values: [0.10 1.00 0.20 0.50]
Final parameter values: [1.22 0.20 0.94 0.58]
mkdir: cannot create directory ‘tssLUZ100’: File exis


Error message was:

*****
***** ERROR: Requested column 4, but database file /home/yostrich/anaconda3/lib/python3.9/site-packages/peaklib/pybedtools.a5ah131e.tmp only has fields 1 - 0.

Error: Unable to open file tssLUZ100/uninfectedControl--NEB_control_spike_in/uninfectedControl--NEB_control_spike_in.minus.peaks.oracle.narrowPeak. Exiting.
mkdir: cannot create directory ‘tssLUZ100’: File exists
INFO:root:Unioning bedgraphs...
INFO:root:Averaging bedgraphs...
INFO:root:/home/yostrich/anaconda3/lib/python3.9/site-packages/peaklib/pybedtools.vxr7lbzj.tmp
INFO:root:Phage_LUZ100_full_genome
INFO:root:tssLUZ100/uninfectedControl-LUZ100_full-genome/uninfectedControl-LUZ100_full-genome.plus.bedgraph
INFO:root:Phage_LUZ100_full_genome
INFO:root:tssLUZ100/uninfectedControl-LUZ100_full-genome/uninfectedControl-LUZ100_full-genome.plus.bedgraph
INFO:root:Phage_LUZ100_full_genome
INFO:root:Merging peaks for /home/yostrich/anaconda3/lib/python3.9/site-packages/peaklib/pybedtools.cn77ogd8.tmp and /h

INFO:root:in-vitro-RNA-transcript
INFO:root:tssLUZ100/uninfectedEnriched-NEB_control_spike_in/uninfectedEnriched-NEB_control_spike_in.plus.bedgraph
INFO:root:in-vitro-RNA-transcript
INFO:root:tssLUZ100/uninfectedEnriched-NEB_control_spike_in/uninfectedEnriched-NEB_control_spike_in.plus.bedgraph
INFO:root:in-vitro-RNA-transcript
INFO:root:Merging peaks for /home/yostrich/anaconda3/lib/python3.9/site-packages/peaklib/pybedtools.ogtfu2k_.tmp and /home/yostrich/anaconda3/lib/python3.9/site-packages/peaklib/pybedtools.hlh3lvde.tmp
INFO:root:Calcluating IDR for /home/yostrich/anaconda3/lib/python3.9/site-packages/peaklib/pybedtools.ogtfu2k_.tmp and /home/yostrich/anaconda3/lib/python3.9/site-packages/peaklib/pybedtools.hlh3lvde.tmp
Initial parameter values: [0.10 1.00 0.20 0.50]
Final parameter values: [1.64 0.20 0.94 0.45]
INFO:root:Unioning bedgraphs...
INFO:root:Averaging bedgraphs...
INFO:root:/home/yostrich/anaconda3/lib/python3.9/site-packages/peaklib/pybedtools.xhb4n5y2.tmp
INFO:root:

In [8]:
# clustering using R
chmod +x peak_clustering.r
./peak_clustering.r "tssLUZ100" 20

Loading required package: dplyr

Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



# TTS detection and clustering

In [ ]:
determinePeaks $BAMFiles ttsLUZ100 "3"
# clustering using R
chmod +x peak_clustering.r
./peak_clustering.r "ttsLUZ100" 10